In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

**Available functions:**

`create_student_df()`: Returns merged studentInfo and studentRegistration with chosen features. Please see dictionaries for map values.

`assessment_namer(id_assessment)`: Helper function for `create_assessment_df()`

`create_assessment_df()`: Returns merged assessments and studentAssessment with chosen features. Currently set to return the first two assessments from courses AAA-FFF.

`wide_form_sa()`: Takes the dataframe returned by `create_assessment_df()` and returns a wide-form variant.

`create_si_sa_df()`: Returns merged dataframes created by `create_student_df()` and `wide_form_sa()`.

`create_vle_df()`: Returns merged vle and studentVle with chosen features. Currently set to return everything on or before day 60 (including pre-course days), binned by 15 day intervals, from courses AAA-FFF.

`wide_form_vle()`: Takes the dataframe returned by `create_vle_df()` and returns a wide-form varient.

`create_si_sa_vle_df()`: Returns merged dataframes created by `create_si_sa_df()` and `wide_form_vle` 




In [2]:
def create_student_df():
    # Read in .csv's to be joined
    studentInfo = pd.read_csv('dataset/studentInfo.csv')
    studentRegistration = pd.read_csv('dataset/studentRegistration.csv')

    gender_map = {'M': 0, 'F': 1}
    ed_map = {'No Formal quals': 0, 'Lower Than A Level': 1, 'A Level or Equivalent': 2, 'HE Qualification': 3, 'Post Graduate Qualification': 4}
    disability_map = {'N': 0, 'Y': 1}
    age_map = {'0-35': 0, '35-55': 1, '55<=': 2}
    result_map = {'Pass': 0, 'Distinction': 0, 'Fail': 1, 'Withdrawn': 2}

    studentInfo["gender"]=studentInfo['gender'].map(gender_map)
    studentInfo["highest_education"]=studentInfo['highest_education'].map(ed_map)
    studentInfo["disability"]=studentInfo['disability'].map(disability_map)
    studentInfo["age_band"]=studentInfo['age_band'].map(age_map)
    studentInfo["final_result"]=studentInfo['final_result'].map(result_map)
    

    # Drop any unnecessary columns (can be edited for feature extraction)
    si_drop_cols = ['region','imd_band']
    si = studentInfo.drop(columns=si_drop_cols) 

    # Merge, inner join
    student_df = pd.merge(si, studentRegistration, how='inner', on=['code_module', 'code_presentation', 'id_student'])

    return student_df

In [3]:
def assessment_namer(id_assessment):
    Assess_1s = [1752, 1758, 14984, 14996, 15008, 15020, 24282, 24291, 25334, 25348, 25355, 25362, 30709, 30714, 30719, 34860, 34873, 34886, 34899]
    Assess_2s = [1753, 1759, 14985, 14997, 15009, 15021, 24283, 24292, 25335, 25349, 25356, 25363, 30710, 30715, 30720, 34861, 34874, 34887, 34900]

    if id_assessment in Assess_1s:
        return "A1"
    elif id_assessment in Assess_2s:
        return "A2"
    else:
        return "Drop"


In [4]:
def create_assessment_df():
    # Read in .csv's to be joined
    assessments = pd.read_csv('dataset/assessments.csv')
    studentAssessment = pd.read_csv('dataset/studentAssessment.csv')

    # Drop any unnecessary columns
    sa_drop_cols = ['is_banked']
    sa = studentAssessment.drop(columns=sa_drop_cols)

    # Merge, left join
    assessment_df = pd.merge(sa, assessments, how='left', on='id_assessment')

    # Dropping 'assessment_type' == 'CMA' due to extreme course differences
    # Also dropping the final exam because we want to catch students BEFORE then
    a_filtered = assessment_df[assessment_df.assessment_type == 'TMA']

    # Mapping with assessment_namer
    a_filtered['assessment_name'] = a_filtered['id_assessment'].apply(assessment_namer)
    a_filtered = a_filtered[a_filtered.assessment_name != 'Drop']

    # Add weighted score column
    a_filtered['weighted_score'] = a_filtered['score'] * a_filtered['weight'] / 100
    # Combine date submitted and date into -/+
    a_filtered['days_from_due'] = a_filtered['date_submitted'] - a_filtered['date']
    

    return a_filtered


In [5]:
def wide_form_sa():
    sa = create_assessment_df()

    # Drop Assessment Type, Weight
    sa.drop(columns=['assessment_type', 'weight', 'date', 'id_assessment'], inplace=True)

    inds = ['id_student', 'code_module', 'code_presentation']
    vals = ['score', 'weighted_score', 'days_from_due', 'date_submitted']

    sa_wide = sa.pivot_table(
                values = vals,
                columns = 'assessment_name',
                index = inds
    )
    
    sa_wide.columns = ["_".join(a) for a in sa_wide.columns.to_flat_index()]
    sa_wide.reset_index(inplace=True)

    sa_wide['score_A1'] = sa_wide['score_A1'].fillna(0)
    sa_wide['score_A2'] = sa_wide['score_A2'].fillna(0)
    sa_wide['weighted_score_A1'] = sa_wide['weighted_score_A1'].fillna(0)
    sa_wide['weighted_score_A2'] = sa_wide['weighted_score_A2'].fillna(0)
    sa_wide['days_from_due_A1'] = sa_wide['days_from_due_A1'].fillna(999)
    sa_wide['days_from_due_A2'] = sa_wide['days_from_due_A2'].fillna(999)
    sa_wide['date_submitted_A1'] = sa_wide['date_submitted_A1'].fillna(999)
    sa_wide['date_submitted_A2'] = sa_wide['date_submitted_A2'].fillna(999)



    return sa_wide


In [6]:
def create_si_sa_df():
    si = create_student_df()
    sa = wide_form_sa()

    # Merge
    si_sa_df = pd.merge(si, sa, how='inner', on=['code_module', 'code_presentation', 'id_student'])

    return si_sa_df

In [7]:
def create_vle_df():
    studentVle = pd.read_csv('dataset/studentVle.csv')
    vle = pd.read_csv('dataset/vle.csv')

    # Dropping unused columns from raw
    vle.drop(columns=['week_from', 'week_to'], inplace=True)

    #Merge - this primarily associates 'id_site' with a more narrative description
    vle_merged = pd.merge(studentVle, vle, how='left', on=['code_module', 'code_presentation', 'id_site'])
    vle_merged.drop(columns='id_site', inplace=True)

    # Dropping GGG
    vle_subset = vle_merged[vle_merged.code_module != 'GGG']

    # Dropping after certain date (Adjust as desired!)
    date_max = 60
    vle_df = vle_subset[vle_subset.date <= date_max]

    # Creating bin columns
    # Set bin parameters (Also adjust as desired!)
    bin_vals = [-15, 0, 15, 30, 45, 60]
    bin_labels = ['pre-0', '1-15', '16-30', '31-45', '46-60']
    vle_df['bin'] = pd.cut(vle_df['date'], bins=bin_vals, labels=bin_labels)
    vle_df.drop(columns='date', inplace=True)

    # Groupby everything but sum_click
    grouper = ['code_module', 'code_presentation', 'id_student', 'activity_type', 'bin']
    vle_df_grouped = vle_df.groupby(grouper)['sum_click'].sum().to_frame()
    vle_df_grouped.reset_index(inplace=True)

    vle_df_grouped['sum_click'] = vle_df_grouped['sum_click'].fillna(0)


    return vle_df_grouped

In [8]:
def wide_form_vle():
    vle_df_long = create_vle_df()
    vle_df_wide = vle_df_long.pivot_table(
                values = 'sum_click',
                columns = ['activity_type', 'bin'],
                index = ['id_student', 'code_module', 'code_presentation']
    )

    vle_df_wide.columns = ["_".join(a) for a in vle_df_wide.columns.to_flat_index()]
    vle_df_wide.reset_index(inplace=True)

    return vle_df_wide


In [9]:
def create_si_sa_vle_df():
    si_sa_df = create_si_sa_df()
    vle = wide_form_vle()

    # Merge
    si_sa_vle_df = pd.merge(si_sa_df, vle, how='inner', on=['code_module', 'code_presentation', 'id_student'])

    return si_sa_vle_df

In [10]:
def create_vle_df_filtered(code_module,code_presentation):
    studentVle = pd.read_csv('dataset/studentVle.csv')
    vle = pd.read_csv('dataset/vle.csv')

    # Prepping Vle info
    # Dropping unused columns from raw vle
    vle.drop(columns=['week_from', 'week_to'], inplace=True)

    #Merge - this primarily associates 'id_site' with a more narrative description
    vle_merged = pd.merge(studentVle, vle, how='left', on=['code_module', 'code_presentation', 'id_site'])
    vle_merged.drop(columns='id_site', inplace=True)

    # For this one, we're lumping all activity_types together.
    vle_merged.drop(columns='activity_type', inplace=True)

    # Filter by module
    vle_subset1 = vle_merged[vle_merged.code_module == code_module]
    # Filter by presentation (semester)
    vle_subset2 = vle_subset1[vle_subset1.code_presentation == code_presentation]

    # Groupby everything but sum_click
    grouper = ['code_module', 'code_presentation', 'id_student', 'date']
    vle_df_grouped = vle_subset2.groupby(grouper)['sum_click'].sum().to_frame()
    vle_df_grouped.reset_index(inplace=True)

    return vle_df_grouped



In [11]:
def wide_form_sa_filtered(code_module, code_presentation):
    sa_wide = wide_form_sa()

    # Filter by module
    sa_subset1 = sa_wide[sa_wide.code_module == code_module]
    # Filter by presentation (semester)
    sa_subset2 = sa_subset1[sa_subset1.code_presentation == code_presentation]

    return sa_subset2

In [20]:
full_df = create_si_sa_vle_df()

In [24]:
def add_submission_bins(full_df):
    #full_df = create_si_sa_vle_df()

    modules = full_df['code_module'].unique()
    presentations = full_df['code_presentation'].unique()

    binned_df = pd.DataFrame(columns=['id_student','code_module', 'code_presentation', 'sum_click_pre_A1', 'sum_click_pre_A2'])

    for m in modules:
        for p in presentations:
            sa_df = wide_form_sa_filtered(m, p)
            vle_df = create_vle_df_filtered(m, p)

            A1_submit = sa_df.set_index('id_student').to_dict()['date_submitted_A1']
            A2_submit = sa_df.set_index('id_student').to_dict()['date_submitted_A2']

        
            vle_df['date_submitted_A1'] = vle_df['id_student'].map(A1_submit)
            vle_df['date_submitted_A2'] = vle_df['id_student'].map(A2_submit)

            vle_df['bin'] = 2

            vle_df.loc[vle_df['date'] < vle_df['date_submitted_A2'], 'bin'] = 'pre_A2'
            vle_df.loc[vle_df['date'] < vle_df['date_submitted_A1'], 'bin'] = 'pre_A1'

            vle_df.fillna(0)
            before_A2 = vle_df[vle_df.bin != 2]

            before_A2.drop(columns=['date_submitted_A1', 'date_submitted_A2'], inplace=True)

            # Groupby
            grouper = ['code_module', 'code_presentation', 'id_student', 'bin']
            vle_df_grouped = before_A2.groupby(grouper)['sum_click'].sum().to_frame()
            vle_df_grouped.reset_index(inplace=True)

            inds = ['id_student', 'code_module', 'code_presentation']
            vals = ['sum_click']

            df_wide = vle_df_grouped.pivot_table(
                        values = vals,
                        columns = 'bin',
                        index = inds
            )

            df_wide.columns = ["_".join(a) for a in df_wide.columns.to_flat_index()]
            df_wide.reset_index(inplace=True)

            binned_df = pd.concat([binned_df, df_wide], ignore_index=True)

    even_fuller_df =  pd.merge(full_df, binned_df, how='inner', on=['code_module', 'code_presentation', 'id_student'])

    return even_fuller_df

In [25]:
really_big_df = add_submission_bins(full_df)


['AAA' 'BBB' 'CCC' 'DDD' 'EEE' 'FFF']
['2013J' '2014J' '2013B' '2014B']
   id_student code_module code_presentation  date_submitted_A1  \
3       11391         AAA             2013J               18.0   

   date_submitted_A2  days_from_due_A1  days_from_due_A2  score_A1  score_A2  \
3               53.0              -1.0              -1.0      78.0      85.0   

   weighted_score_A1  weighted_score_A2  
3                7.8               17.0  
  code_module code_presentation  id_student  date  sum_click
0         AAA             2013J       11391    -5         98
   id_student code_module code_presentation  sum_click_pre_A1  \
0       11391         AAA             2013J             352.0   

   sum_click_pre_A2  
0             175.0  
   id_student code_module code_presentation  date_submitted_A1  \
0        6516         AAA             2014J               17.0   

   date_submitted_A2  days_from_due_A1  days_from_due_A2  score_A1  score_A2  \
0               51.0              -2.0  

,code_module,code_presentation,id_student,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,final_result,...,subpage_16-30,subpage_31-45,subpage_46-60,url_pre-0,url_1-15,url_16-30,url_31-45,url_46-60,sum_click_pre_A1,sum_click_pre_A2
0,AAA,2013J,11391,0,3,2,0,240,0,0,...,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,352.0,175.0
1,AAA,2013J,28400,1,3,1,0,60,0,0,...,16.0,1.0,1.0,10.0,9.0,11.0,1.0,1.0,493.0,156.0
2,AAA,2013J,31604,1,2,1,0,60,0,0,...,17.0,17.0,3.0,3.0,10.0,8.0,12.0,3.0,390.0,371.0
3,AAA,2013J,32885,1,1,0,0,60,0,0,...,8.0,0.0,1.0,3.0,0.0,3.0,0.0,0.0,558.0,64.0
4,AAA,2013J,38053,0,2,1,0,60,0,0,...,11.0,3.0,3.0,1.0,4.0,6.0,2.0,3.0,614.0,395.0


In [26]:
really_big_df.to_excel("wideform.xlsx", index=False)